# Part 2: Data Preparation
 ----

Note this Demo is based on https://github.com/pytorch/vision/tree/v0.11.3

This notebook walks you each step to train a model using containers from the NGC Catalog. We chose the GPU optimized Pytorch container as an example. The basics of working with docker containers apply to all NGC containers.

We will show you how to:

* Download the Xview Dataset
* How to convert labels to coco format
* How to conduct the preprocessing step ,Tiling: slicing large satellite imagery into chunks 
* How to upload to s3 bucket to support distributed training

Let's get started!

---


### 2. Download the TensorFlow container from the NGC Catalog 

Once the Docker Engine is installed on your machine, visit https://ngc.nvidia.com/catalog/containers and search for the TensorFlow container. Click on the TensorFlow card and copy the pull command.
UPDATE IMG




# 1. Download the Xview Dataset
The dataset we will be using is from the DIUx xView 2018 Challenge https://challenge.xviewdataset.org by U.S. National Geospatial-Intelligence Agency (NGA). You will need to create an account at https://challenge.xviewdataset.org/welcome, agree to the terms and conditions, and download the dataset manually.

You can download the dataset at the url https://challenge.xviewdataset.org/data-download



In [3]:
# run pip install to get the SAHI library
!pip install sahi scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 153.5 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 191.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 161.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 188.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 172.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 188.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


In [ ]:
# Example command to download train images with wget command, you will need to update the url as the token is expired"
!wget -O train_images.tgz "https://d307kc0mrhucc3.cloudfront.net/train_images.tgz?Expires=1680826141&Signature=par-WmB2Ffj-BZUoVkApKYADOMnqEQMWo7ZalDo47UqhPvy0nDge6pTUaEYH8F7xSR8nJKb3fFHdfqFea9Jua5LgqTa1sp5Ekaw8FloYIJIFvv-S0OxA-5VRpyYNLiKNjIg4uxykSKMYPj3xTq8YicBZNdnrXzafsRxmeQmcbiSqGR~8Jf1PndguouuNa4TV0D4iBtKqF0G6phgNCCF3ofGXO6YwLjjaVyKVsyLcvuQ2xj4KGKJM0AP2VJA43XjlgGJEuNzh1LubPocN8OCUlbm~jnUxq1N0iWDBFfxdW2JhCFN-iajIsgWEzQd0SMxB7bpyQhfWNEFHwNcA71uwKQ__&Key-Pair-Id=APKAIKGDJB5C3XUL2DXQ"

In [ ]:
# Example command to download train images with wget command, you will need to update the url as the token is expired"
!wget -O train_labels.tgz "https://d307kc0mrhucc3.cloudfront.net/train_labels.tgz?Expires=1680826141&Signature=ToOHVlmZq6tjN0La0wYL9~DEeaf9HK1F0KB8yy4Izk020HJemSDzakYmhCF3CsXJ3ns-KrZ4Vfws6mIlmfkk9l0FvVByQC94MB618CaRBytbCkO69ONUFAt0OzNUR14DB9cCM6Q3VJ9dHcUw-fAr~D2yHeK3mhnDSNbCQAqOaKoYQlfLxbAdTrMfU8KL6z3vAD6hC0ofa6QtlSxbhJgXupfw7nzgNmtrhF2Q6xX7gDSoi6~7OLu7bisGlA8sJuzPCVONWl5zxwK~ZPOgJsF6UAckdKsd2V-4IX4cWQlZYUf7FkKG3ccT8XFmeHY-9dBfX2AgfFH84p5P~nrPA41R-Q__&Key-Pair-Id=APKAIKGDJB5C3XUL2DXQ"

In [ ]:
# unzip images and labels from /run/determined/workdir directory 
!tar -xvf train_images.tgz -C e2e_blogposts/ngc_blog/xview_dataset/

In [ ]:
# unzip labels from /run/determined/workdir directory 
!tar -xvf train_labels.tgz -C e2e_blogposts/ngc_blog/xview_dataset/

# Convert TIF to RGB

In [21]:
# Here loop through all the images and convert them to RGB, this is important for tiling the images and training with pytorch
# will take about an hour to complete
!python data_utils/tif_2_rgb.py --input_dir xview_dataset/train_images \
  --out_dir xview_dataset/train_images_rgb/

renaming bad named files...
[PosixPath('100.tif'), PosixPath('109.tif'), PosixPath('102.tif')]
100%|███████████████████████████████████████| 846/846 [1:03:55<00:00,  4.53s/it]


# 2. How to convert labels to coco format
Here we run a script to convert the dataset labels from .geojson format to COCO format. More details on the COCO format here: 

The result will be two files (in COCO formal) generated `train.json` and `val.json`

In [11]:
# make sure train_images_dir is pointing to the .tif images
!python data_utils/convert_geojson_to_coco.py --train_images_dir xview_dataset/train_images/ \
  --train_images_dir_rgb xview_dataset/train_images_rgb/ \
  --train_geojson_path xview_dataset/xView_train.geojson \
  --output_dir xview_dataset/ \
  --train_split_rate 0.75 \
  --category_id_remapping data_utils/category_id_mapping.json \
  --xview_class_labels data_utils/xview_class_labels.txt


Namespace(category_id_remapping='data_utils/category_id_mapping.json', output_dir='xview_dataset/', train_geojson_path='xview_dataset/xView_train.geojson', train_images_dir='xview_dataset/train_images/', train_images_dir_rgb='xview_dataset/train_images_rgb/', train_split_rate=0.75, xview_class_labels='data_utils/xview_class_labels.txt')
5.tif:  True
Parsing xView data: 100%|████████████| 601937/601937 [00:14<00:00, 42033.53it/s]
chips:  ['2355.png' '2355.png' '2355.png' ... '389.png' '389.png' '389.png']
Converting xView data into COCO format: 100%|█| 846/846 [01:12<00:00, 11.64it/s]


# 3. Slicing/Tiling the Dataset
Here we are using the SAHI library to slice our large satellite images. Satellite images can be up to 50k^2 pixels in size, which wouldnt fit in GPU memory. We alleviate this problem by slicing the image. 

In [ ]:
!python data_utils/slice_coco.py --image_dir xview_dataset/train_images_rgb/ \
  --dataset_json_path xview_dataset/train.json \
  --slice_size 300 \
  --overlap_ratio 0.2 \
  --ignore_negative_samples True \
  --min_area_ratio 0.1 \
  --output_dir xview_dataset/train_images_rgb_no_neg/

Slicing step is starting...
indexing coco dataset annotations...
 50%|████████████████████▎                    | 314/634 [23:02<21:57,  4.12s/it]

# 4. How to upload to s3 bucket to support distributed training